In [86]:
from PIL import Image,ImageOps,ImageStat,ImageDraw
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow  import keras as keras
from keras import Model
from keras import layers
from keras import losses
from keras.models import Sequential
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Activation, Conv2D, add, Concatenate
#from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import  ModelCheckpoint
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from keras.optimizers import Adam
from keras.utils import to_categorical
import os,os.path
import sys
import math
import sys
import numpy as np
import random

psp_fileName = "D:/image_data_boundaries/__FR1_modified.jpg._PSP"
image_fileName = "D:/image_data_boundaries/__FR1_modified.jpg"
newMask_fileName = "D:/image_data_boundaries/new_mask_class_"
res_filename_winsize = "D:/image_data_boundaries/homogen_parts"
fragm_dir = "D:/image_data_boundaries/fragments"
param1 = 80
#acc_file = 'D:/_SAR_Kubinka/accuracy.txt'
result_fileName = "D:/image_data_boundaries/classification_result"
#marks_filename = 'D:/_SAR_Kubinka/class_marks.txt'
#img_len = 1024

img_file_name = image_fileName
#дополнять массив цветами по мере необходимости\n",
t_color = ['red', 'blue']
temple_file = Image.open(img_file_name)
colors = {}
#сборка границ прямоугольникв, из которых будут вырезаться области (если их на изображении несколько)
classRects=[]
curColor = ""
class_numb = '021'
rect_flag  = False
   
img = Image.new('RGB', temple_file.size, color=0)
draw  = ImageDraw.Draw(img)
fp = open(psp_fileName)
print("all was opened")
for k, txt in enumerate(fp):
    if k<2:
        continue
    zz, t = txt.split('=')
    if t.find('Pline') >= 0:
        x2y = []
        rect_flag = False
        continue
    if t.find('Rectangle') >= 0:
        x2y = []
        rect_flag = True
        continue
    if t.find('Pen') >= 0:
        p = t.split(',')
        curColor = p[2]
        if p[2] not in  colors:
            colors.update({p[2]: len(colors)})
   
    t = t.split(' ')
    if t[0] == '':
        if rect_flag:
            x2y.insert(1,(x2y[0][0],x2y[1][1]))
            x2y.append( (x2y[2][0], x2y[0][1]))  
        idx = colors.get(curColor) 
        if idx ==  len(classRects):
            classRects.append([x2y])
        else:
            classRects[idx].append(x2y)
        #print(classRects)   
        draw.polygon(x2y, outline = t_color[idx], fill = t_color[idx])
        continue
    x2y.append((int(t[0]), int(t[1])))
print("masks were drawen")
img.save(newMask_fileName + class_numb +".BMP")
img.close()
fp.close()

all was opened
masks were drawen


In [87]:
win_size = 16
step_len = 8
bound = math.ceil(math.sqrt(2)*win_size-win_size)
dir_name = fragm_dir
labels=[]
nmb_fragm = 0
print(bound)
im = Image.open(img_file_name)
for i in range(len(colors)):
    for j in range(len(classRects[i])):
        
        x0 = classRects[i][j][0][0]-step_len
        y0 = classRects[i][j][0][1]-step_len
        x_len =  classRects[i][j][1][1]-classRects[i][j][0][1]+step_len
        y_len =  classRects[i][j][2][0]-classRects[i][j][1][0]+step_len
        x_amount = int(x_len/step_len)
        y_amount = int(y_len/step_len)
        for k in range(x_amount):
            for l in range(y_amount):
                labels.append(i)
               
                im2=im.crop((x0 + k * step_len -bound, y0+ l * step_len -bound, x0 + k * step_len +bound + win_size,y0+ l * step_len +bound + win_size)) 
                imfnam=dir_name+"\\+++fragm{:05d}.jpg".format(nmb_fragm); 
                nmb_fragm+=1;
                im2.save(imfnam)
im.close()

7


In [88]:
gen_crop_img = True
nmb_fragm = 0
print(nmb_fragm)
if gen_crop_img:
    img = Image.open(img_file_name)
    width, height = img.size
    print(width, height)
    with open ("D:/image_data_boundaries/list_of_fragments.txt", 'wt') as file:
        for i in range(0, width, step_len):
            for j in range(0, height, step_len):
                if(i + win_size) < width and (j + win_size) < height:
                    x2y = [(i, j), (i + win_size, j), (i + win_size, j + win_size), (i, j + win_size)]
                    im2 = img.crop((i, j, i + win_size, j + win_size))
                    imfnam = "D:\\image_data_boundaries\\full_img\\+++fragm{:05d}.jpg".format(nmb_fragm)
                    file.write(imfnam + " " + str(i) + " " + str(j) + "\n")
                    nmb_fragm += 1;
                    im2.save(imfnam)
    img.close()


0
858 856


In [89]:
load_in_list = True
if load_in_list:
    img_list = []
    img_coords=[]
    flag_read = True
    with open("D:\\image_data_boundaries\\list_of_fragments.txt", 'r') as file:
        for bf in file:
            bf_list = bf.split(" ")
            img_list.append(bf_list[0])
            #print(img_list)
            img_coords.append((int(bf_list[1]), int(bf_list[2])))
            #print(bf)

In [90]:
transAmount = 36;
dlin   = 0

def load_CNN_train_augment(dir_name1):
    ll=[]           #пустой список имен входных файлов JPG 
    for file in os.listdir(dir_name1):
        if file.endswith(".jpg"): ll.append(file)
    dlin   = len(ll)     
    dlin0  = int(len(ll)/2)  #будем пропускать нечетные элементы
    #dlin=int(dlin0*(var+1) )  #0 ->просто чтение, 1 ->+90, 2 ->+180, 3 ->+270 дополняем массив
    train_x = np.zeros((dlin * transAmount, win_size, win_size, 3), dtype='float32')
    print('train_x_augm.shape=',train_x.shape)
    train_z = np.zeros((dlin * transAmount, 1), dtype='float32')
    dlin = dlin * transAmount
    k=0
    for file in ll:
        im  = Image.open(dir_name+"\\"+file)
        
        shift0 = random.randint(0,7)
        shift1 = random.randint(0,7)
        shift2 = random.randint(0,7)
        shift3 = random.randint(0,7)
        #shift0 = 3
        #shift1 = 3
        #shift2 = 3
        #shift3 = 3
        '''
        if shift0 == 0 and shift1 == 0:
            shift0 += 2
        if shift2 == 0 or shift2 == 3:    
            shift2 +=1
        if shift3 == 0 or shift3 == 3:    
            shift3 +=1    
        '''
       
        train_x[transAmount * k + 1] =np.array(im.crop((bound,bound,bound + win_size, bound +win_size)))*1./255.
        train_x[transAmount * k + 2] =np.array(im.rotate(90).crop((bound,bound,bound + win_size, bound +win_size)))*1./255.
        train_x[transAmount * k + 3] =np.array(im.rotate(45).crop((bound,bound,bound + win_size, bound +win_size)))*1./255.
        train_x[transAmount * k + 4] =np.array(im.rotate(30).crop((bound,bound,bound + win_size, bound +win_size)))*1./255.
        train_x[transAmount * k + 5] =np.array(im.rotate(75).crop((bound,bound,bound + win_size, bound +win_size)))*1./255.
        train_x[transAmount * k + 6] =np.array(im.rotate(180).crop((bound,bound,bound + win_size, bound +win_size)))*1./255.
        
        train_x[transAmount * k + 7] =np.array(im.crop((shift0,shift0,shift0 + win_size, shift0 +win_size)))*1./255.
        train_x[transAmount * k + 8] =np.array(im.rotate(90).crop((shift0,shift0,shift0 + win_size, shift0 +win_size)))*1./255.
        train_x[transAmount * k + 9] =np.array(im.rotate(45).crop((shift0,shift0,shift0 + win_size, shift0 +win_size)))*1./255.
        train_x[transAmount * k + 10] =np.array(im.rotate(30).crop((shift0,shift0,shift0 + win_size, shift0 +win_size)))*1./255.
        train_x[transAmount * k + 11] =np.array(im.rotate(75).crop((shift0,shift0,shift0 + win_size, shift0 +win_size)))*1./255.
        train_x[transAmount * k + 12] =np.array(im.rotate(180).crop((shift0,shift0,shift0 + win_size, shift0 +win_size)))*1./255.
        
        train_x[transAmount * k + 13] =np.array(im.crop((shift1,shift1,shift1 + win_size, shift1 +win_size)))*1./255.
        train_x[transAmount * k + 14] =np.array(im.rotate(90).crop((shift1,shift1,shift1 + win_size, shift1 +win_size)))*1./255.
        train_x[transAmount * k + 15] =np.array(im.rotate(45).crop((shift1,shift1,shift1 + win_size, shift1 +win_size)))*1./255.
        train_x[transAmount * k + 16] =np.array(im.rotate(30).crop((shift1,shift1,shift1 + win_size, shift1 +win_size)))*1./255.
        train_x[transAmount * k + 17] =np.array(im.rotate(75).crop((shift1,shift1,shift1 + win_size, shift1 +win_size)))*1./255.
        train_x[transAmount * k + 18] =np.array(im.rotate(180).crop((shift1,shift1,shift1 + win_size, shift1 +win_size)))*1./255.
        
        train_x[transAmount * k + 19] =np.array(im.crop((shift2,shift2,shift2 + win_size, shift2 +win_size)))*1./255.
        train_x[transAmount * k + 20] =np.array(im.rotate(90).crop((shift2,shift2,shift2 + win_size, shift2 +win_size)))*1./255.
        train_x[transAmount * k + 21] =np.array(im.rotate(45).crop((shift2,shift2,shift2 + win_size, shift2 +win_size)))*1./255.
        train_x[transAmount * k + 22] =np.array(im.rotate(30).crop((shift2,shift2,shift2 + win_size, shift2 +win_size)))*1./255.
        train_x[transAmount * k + 23] =np.array(im.rotate(75).crop((shift2,shift2,shift2 + win_size, shift2 +win_size)))*1./255.
        train_x[transAmount * k + 24] =np.array(im.rotate(180).crop((shift2,shift2,shift2 + win_size, shift2 +win_size)))*1./255.
       
        train_x[transAmount * k + 25] =np.array(im.crop((shift3,shift3,shift3 + win_size, shift3 +win_size)))*1./255.
        train_x[transAmount * k + 26] =np.array(im.rotate(90).crop((shift3,shift3,shift3 + win_size, shift3 +win_size)))*1./255.
        train_x[transAmount * k + 27] =np.array(im.rotate(45).crop((shift3,shift3,shift3 + win_size, shift3 +win_size)))*1./255.
        train_x[transAmount * k + 28] =np.array(im.rotate(30).crop((shift3,shift3,shift3 + win_size, shift3 +win_size)))*1./255.
        train_x[transAmount * k + 29] =np.array(im.rotate(75).crop((shift3,shift3,shift3 + win_size, shift3 +win_size)))*1./255.
        train_x[transAmount * k + 0] =np.array(im.rotate(180).crop((shift3,shift3,shift3 + win_size, shift3 +win_size)))*1./255.
        
        train_x[transAmount * k + 30] =np.array(im.crop((shift3,0,shift3 + win_size, 0 +win_size)))*1./255.
        train_x[transAmount * k + 31] =np.array(im.rotate(90).crop((shift3,0,shift3 + win_size, 0 +win_size)))*1./255.
        train_x[transAmount * k + 32] =np.array(im.rotate(45).crop((shift3,0,shift3 + win_size, 0 +win_size)))*1./255.
        train_x[transAmount * k + 33] =np.array(im.rotate(30).crop((shift3,0,shift3 + win_size, 0 +win_size)))*1./255.
        train_x[transAmount * k + 34] =np.array(im.rotate(75).crop((shift3,0,shift3 + win_size, 0 +win_size)))*1./255.
        train_x[transAmount * k + 35] =np.array(im.rotate(180).crop((shift3,0,shift3 + win_size, 0 +win_size)))*1./255.
    #train_x[transAmount * k + 4] = np.array(im.crop((0,8,13,21)))*1./255.   
        for i in range(transAmount):
            train_z[k * transAmount + i] = int(labels[k])
        
        k+=1
        if k==dlin: break
    temp=[]
    for i in range(len(train_z)):
        temp.append(to_categorical(train_z[i],num_classes=len(colors)))
    train_y=np.array(temp)    
    return train_x,train_y

train_x1,train_y1=load_CNN_train_augment(dir_name)

train_x_augm.shape= (23292, 16, 16, 3)


In [91]:
amount_valid = 1500
k = 0
idx_del = set()
train_x_valid = np.zeros((amount_valid, win_size, win_size, 3), dtype='float32')
train_y_valid = np.zeros((amount_valid,1,2), dtype='float32')
while k < amount_valid:
    idx = random.randint(0, train_x1.shape[0]-1)
    while idx in idx_del:
        idx = random.randint(0, train_x1.shape[0]-1)
    idx_del.add(idx)
    train_x_valid[k] = train_x1[idx]
    train_y_valid[k] = train_y1[idx]
    k += 1



idx_del = list(idx_del)
train_x_data1 = np.delete(train_x1, idx_del,0)
train_y_data1 = np.delete(train_y1, idx_del,0)

print('train_x_augm.shape=',train_y_valid.shape)
print('train_x_augm.shape=',train_x_data1.shape)
#реорганизация набора тестовых данных, чтобы была равномерность 
train_x_valid_buf = np.zeros((train_x_data1.shape[0]//2, win_size, win_size, 3), dtype='float32')
train_y_valid_buf = np.zeros((train_x_data1.shape[0]//2,1,2), dtype='float32')

for i in range(train_x_data1.shape[0]//2):
    if i%2 ==0:
        train_x_valid_buf[i] = train_x_data1[2*i]
        train_y_valid_buf[i] = train_y_data1[2*i]
np.flip( train_x_valid_buf)  
np.flip( train_y_valid_buf)  
for i in range(train_x_data1.shape[0]//2):
    if i%2 ==0:
        train_x_data1[2*i] = train_x_valid_buf[i]
        train_y_data1[2*i] = train_y_valid_buf[i]
train_x_data = np.zeros((train_x1.shape[0],win_size, win_size, 3), dtype='float32')  
train_y_data = np.zeros((train_y1.shape[0],1,2), dtype='float32')  
np.concatenate(( train_x_data1,train_x_valid),  out = train_x_data)    
np.concatenate(( train_y_data1,train_y_valid),  out = train_y_data)    

train_x_augm.shape= (1500, 1, 2)
train_x_augm.shape= (21792, 16, 16, 3)


array([[[1., 0.]],

       [[1., 0.]],

       [[1., 0.]],

       ...,

       [[1., 0.]],

       [[0., 1.]],

       [[1., 0.]]], dtype=float32)

In [92]:
num_train, height_et, width_et, depth_et = train_x_data.shape

In [93]:
def def_cnn_model_4(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 30#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_5)
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_7)
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
    
    conv_9 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_4)
    conv_10 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_9)
   # pool_5 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_10)
    drop_5 = Dropout(drop_prob_1)(conv_10)
    flat = Flatten()(drop_5)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [94]:
def def_cnn_model_0(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 25#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
    x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , x1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
    x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_5)
    conv_6 =add([conv_6 , x2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , x3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
   
    
    flat = Flatten()(drop_4)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [95]:
def def_cnn_model_1(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 30#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
    x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
    conv_4 =add([conv_4, x1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
    x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_5)
    conv_6 =add([conv_6 , x2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , x3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
    
    
    flat = Flatten()(drop_4)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [96]:
def def_cnn_model_2(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 20#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
    x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , x1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
    x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_5)
    conv_6 =add([conv_6 , x2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , x3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
    
    
    flat = Flatten()(drop_4)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [97]:
def train_CNN_model_ex(model_t,train_xt,train_yt,dlin_t,msize,step1,step2,val_split):
    model_temp=model_t
    val_acc_min=0.0
    for k in range(step1):
        #model_t.fit(train_xt,train_yt,epochs=step2,verbose=0,batch_size=16,validation_split=0.15)
        hist=model_t.fit(train_xt,train_yt.squeeze(),epochs=1,verbose=0,validation_split = val_split, shuffle = True)#, callbacks=[mcp_save])
        #print('First layer weights:', model.get_weights()[0])
        vvv=hist.history['val_accuracy']
        #vvv=hist.history['val_acc']
        print(k,vvv)
        val_acc = vvv[0]
        if val_acc > val_acc_min:
            model_temp = model_t
            val_acc_min=val_acc
            hist=model_t.fit(x=train_xt,y=train_yt.squeeze(),epochs=1,verbose=1,validation_split=val_split, shuffle = True, batch_size=64)#, callbacks=[mcp_save])#, callbacks=[mcp_save])
        
    #get_CNN_stat(train_xt,train_yt,dlin_t,model_t,msize)
    #get_CNN_stat(train_xz,train_yz,dlin_t,model_t,msize)
    print('end CNN train')
    return model_temp

In [98]:
def def_cnn_model_5(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 25#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
   # x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , drop_1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
   # x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_5)
    #conv_6 =add([conv_6 , drop_2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    #x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , drop_3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
   
    
    flat = Flatten()(drop_4)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [99]:
def def_cnn_model_6(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 25#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
    x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , x1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
    x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_5)
    conv_6 =add([conv_6 , x2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , x3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
   
    x4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_4)
    conv_9 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_4)
    conv_10 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
    conv_10 =add([conv_10 , x4])
    
    drop_5 = Dropout(drop_prob_1)(conv_10 )
    flat = Flatten()(drop_5)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(input=inp, output=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [100]:
def def_cnn_model_7(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 20#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
   # x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , drop_1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
   # x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    #conv_6 =add([conv_6 , drop_2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    #x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , drop_3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
   
    
    flat = Flatten()(drop_4)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [101]:
def def_cnn_model_8(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 30#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size),padding='same')(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
   # x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , drop_1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size),padding='same')(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
   # x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    #conv_6 =add([conv_6 , drop_2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size),padding='same')(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    #x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , drop_3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size),padding='same')(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
   
    
    flat = Flatten()(drop_4)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [102]:
def def_cnn_model_9(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 20#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
   # x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , drop_1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
   # x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='tanh')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    #conv_6 =add([conv_6 , drop_2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    #x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='tanh')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , drop_3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
   
    
    flat = Flatten()(drop_4)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [103]:
def def_cnn_model_10(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 20#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
   # x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , drop_1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
   # x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    #conv_6 =add([conv_6 , drop_2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    #x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , drop_3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
    
    conv_9 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_4)
    conv_10 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_9)
    conv_10 =add([conv_8 , drop_4])
    #pool_5 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_10)
    drop_5 = Dropout(drop_prob_1)(conv_10)
    
    flat = Flatten()(drop_5)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [104]:
def def_cnn_model_11(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 20#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
   # x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , drop_1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
   # x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    #conv_6 =add([conv_6 , drop_2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    #x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , drop_3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
    
    conv_9 = Convolution2D(conv_depth_1-0, kernel_size, kernel_size, padding='same', activation='relu')(drop_4)
    conv_10 = Convolution2D(conv_depth_1-0, kernel_size, kernel_size, padding='same', activation='relu')(conv_9)
    #conv_10 =add([conv_8 , drop_4])
    #pool_5 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_10)
    drop_5 = Dropout(drop_prob_1)(conv_10)
    
    flat = Flatten()(drop_5)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [105]:
def def_cnn_model_12(height,width,depth):
    batch_size = 8 # in each iteration, we consider 32 training examples at once
    num_epochs = 2 # we iterate 200 times over the entire training set
    kernel_size = 3 # we will use 3x3 kernels throughout
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 25#height*2 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = height #64 # ...switching to 64 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 68 # the FC layer will have 512 neurons
    num_classes=2

    inp = Input(shape=(height, width,depth)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
    conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='tanh')(inp)
    conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
    pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    
   # x1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
    conv_3 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(drop_1)
    conv_4 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_3)
    conv_4 =add([conv_4 , drop_1])
    pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_4)
    drop_2 = Dropout(drop_prob_1)(pool_2)
    
    
   # x2 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_2)
    conv_5 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_2)
    conv_6 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_5)
    #conv_6 =add([conv_6 , drop_2])
    pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_6)
    drop_3 = Dropout(drop_prob_1)(pool_3)
    
    #x3 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_3)
    conv_7 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(drop_3)
    conv_8 = Convolution2D(conv_depth_1-10, kernel_size, kernel_size, padding='same', activation='relu')(conv_7)
    conv_8 =add([conv_8 , drop_3])
    pool_4 = MaxPooling2D(pool_size = (pool_size, pool_size), padding='same')(conv_8)
    drop_4 = Dropout(drop_prob_1)(pool_4)
    
    conv_9 = Convolution2D(conv_depth_1-0, kernel_size, kernel_size, padding='same', activation='relu')(drop_4)
    conv_10 = Convolution2D(conv_depth_1-0, kernel_size, kernel_size, padding='same', activation='relu')(conv_9)
    #conv_10 =add([conv_8 , drop_4])
    #pool_5 = MaxPooling2D(pool_size = (pool_size, pool_size))(conv_10)
    drop_5 = Dropout(drop_prob_1)(conv_10)
    
    flat = Flatten()(drop_5)
   
    #flat= pool_1
    #out = Dense(hidden_size, activation='tanh')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
    #dense1 = Dense(20, activation='sigmoid')(flat)
    out = Dense(num_classes, activation='softmax')(flat)

    model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy
    return model

In [106]:
def classify_img_by_batch(model, filename_suffix, res_fname_suffix):
    temple_file = Image.open(img_file_name)
    img = Image.new('RGB', temple_file.size, color=0)
    draw = ImageDraw.Draw(img)
    width, height = img.size
    class_marks = np.zeros((width, height), dtype='int')
    batch_size_t = 32
    batch_count = 0
    batch_pack = np.zeros((batch_size_t , win_size, win_size, 3), dtype='float32')
    batch_coords = []
    idx_counter = 0
    with open(res_filename_winsize + res_fname_suffix, 'w') as file:
        for i in img_list:


            if(batch_count < batch_size_t ):
                im = Image.open(i)

                #print(img_coords[idx_counter])
                batch_pack[batch_count] = np.array(im)*1./255.
                #print(batch_pack[batch_count])
                batch_coords.append(img_coords[idx_counter])
                batch_count += 1
                idx_counter += 1
                im.close()
            else:
                #pred_res = model.predict_on_batch(batch_pack)
                pred_res = model.predict(batch_pack,batch_size=batch_size_t )
                print(pred_res)
                for j in range(pred_res.shape[0]):
                    i_idx = batch_coords[j][0]
                    j_idx = batch_coords[j][1]
                    max_idx = np.argmax(pred_res[j])
                    #if max_idx == 0:
                       # max_idx = 1
                   # else:
                        #max_idx = 0
                    file.write(str(i_idx) + " " + str( height - j_idx ) + " " + str(max_idx) + "\n")
                    for k in range(win_size):
                        for l in range(win_size):
                            class_marks[i_idx + k, j_idx + l] = max_idx
                    draw.polygon([(i_idx, j_idx), (i_idx + win_size, j_idx), (i_idx + win_size, j_idx+ win_size), (i_idx, j_idx + win_size)], outline=t_color[max_idx], fill=t_color[max_idx])
                batch_count = 0
                batch_coords = []
                im = Image.open(i)
                #print(i)
                #print(img_coords[idx_counter])
                batch_pack[batch_count] = np.array(im)*1./255.
                batch_coords.append(img_coords[idx_counter])
                batch_count += 1
                idx_counter += 1
                im.close()
    file.close()  
    img.save(result_fileName + filename_suffix)
    img.close()
    return class_marks

def classify_flat(model, filename_suffix):
    temple_file = Image.open(img_file_name)
    img = Image.new('RGB', temple_file.size, color=0)
    x0 = 0
    y0 = 0
    draw  = ImageDraw.Draw(img)
    width, height = img.size
    im2 = np.zeros((1, win_size, win_size, 3), dtype='float32')
    class_marks = np.zeros((width,height), dtype='int')
    for i in range(0, width,step_len ):
        for j in range(0, height,step_len):
            if(i + win_size) < width and (j + win_size ) < height:
                x2y = [(i,j), (i + win_size, j),(i + win_size, j + win_size), (i, j + win_size)]
                im2[0] = np.array(  temple_file.crop((i, j, i + win_size,j + win_size)))*1./255.
                z = model1.predict(im2)
                m_idx = np.argmax(z[0])
                for k in range(win_size):
                    for l in range(win_size):
                        class_marks[i + k, j + l] = m_idx
                draw.polygon(x2y, outline = t_color[m_idx], fill = t_color[m_idx])

    img.save(result_fileName + filename_suffix)
    img.close()
    return class_marks

In [107]:
def classify_img_by_batch_res_UDP(model, filename_suffix, res_fname_suffix):
    temple_file = Image.open(img_file_name)
    img = Image.new('RGB', temple_file.size, color=0)
    draw = ImageDraw.Draw(img)
    width, height = img.size
    class_marks = np.zeros((width, height), dtype='int')
    batch_size_t = 32
    batch_count = 0
    batch_pack = np.zeros((batch_size_t , win_size, win_size, 3), dtype='float32')
    batch_coords = []
    idx_counter = 0
    homogen_parts = {}
    with open(res_filename_winsize + res_fname_suffix, 'w') as file:
        for i in img_list:
            if(batch_count < batch_size_t ):
                im = Image.open(i)
                batch_pack[batch_count] = np.array(im)*1./255.
                batch_coords.append(img_coords[idx_counter])
                batch_count += 1
                idx_counter += 1
                im.close()
            else:
                pred_res = model.predict(batch_pack,batch_size=batch_size_t )
                
                for j in range(pred_res.shape[0]):
                    i_idx = batch_coords[j][0]
                    j_idx = batch_coords[j][1]
                   #print(i_idx, j_idx)
                    max_idx = np.argmax(pred_res[j])
                    #if max_idx == 0:
                       # max_idx = 1
                   # else:
                        #max_idx = 0
                    homogen_parts[(i_idx,height - j_idx)]=max_idx
                    #file.write(str(i_idx) + " " + str( height - j_idx ) + " " + str(max_idx) + "\n")
                    #for k in range(win_size):
                        #for l in range(win_size):
                            #class_marks[i_idx + k, j_idx + l] = max_idx
                    #draw.polygon([(i_idx, j_idx), (i_idx + win_size, j_idx), (i_idx + win_size, j_idx+ win_size), (i_idx, j_idx + win_size)], outline=t_color[max_idx], fill=t_color[max_idx])
                batch_count = 0
                batch_coords = []
                im = Image.open(i)
                #print(i)
                #print(img_coords[idx_counter])
                batch_pack[batch_count] = np.array(im)*1./255.
                batch_coords.append(img_coords[idx_counter])
                batch_count += 1
                idx_counter += 1
                im.close()
        for i in homogen_parts:
            i_idx = i[0]
            j_idx = height - i[1]
            elem_amount = 0
            summ = 0
            
            if (i_idx-step_len, j_idx) in homogen_parts:
                elem_amount += 1
                summ += homogen_parts[(i_idx-step_len, j_idx)]
            if (i_idx + step_len, j_idx) in homogen_parts:
                elem_amount += 1
                summ += homogen_parts[(i_idx + step_len, j_idx)]
            if (i_idx, j_idx-step_len) in homogen_parts:
                elem_amount += 1
                summ += homogen_parts[(i_idx, j_idx-step_len)]
            if (i_idx , j_idx+ step_len) in homogen_parts:
                elem_amount += 1
                summ += homogen_parts[(i_idx , j_idx + step_len)]

            if elem_amount == 4:
                if summ == 0 or summ == 1:
                    homogen_parts.update({(i_idx, j_idx): 0})
                if summ == 3 or summ == 4:
                    homogen_parts.update({(i_idx, j_idx): 1})
            if elem_amount == 3:
                if summ == 0 or summ == 1:
                    homogen_parts.update({(i_idx, j_idx): 0})
                if summ == 3 :
                    homogen_parts.update({(i_idx, j_idx): 1})
            
        #print(homogen_parts)
        #print( homogen_parts.get((0, 0)))
        for i in homogen_parts:
            i_idx = i[0]
            j_idx = height - i[1]
            file.write(str(i_idx) + " " + str(  i[1] ) + " " + str(homogen_parts[(i_idx,  i[1])]) + "\n")
            for k in range(win_size):
                for l in range(win_size):
                    class_marks[i_idx + k, j_idx + l] = homogen_parts[(i_idx,  i[1])]
            draw.polygon([(i_idx, j_idx), (i_idx + win_size, j_idx), (i_idx + win_size, j_idx+ win_size), (i_idx, j_idx + win_size)], outline=t_color[homogen_parts[(i_idx,  i[1])]], fill=t_color[homogen_parts[(i_idx,  i[1])]])
    file.close()
    img.save(result_fileName + filename_suffix)
    img.close()
    return class_marks

In [108]:
def classify_img_by_batch_res_UDP_modified(model, filename_suffix, res_fname_suffix):
    temple_file = Image.open(img_file_name)
    img = Image.new('RGB', temple_file.size, color=0)
    draw = ImageDraw.Draw(img)
    width, height = img.size
    class_marks = np.zeros((width, height), dtype='int')
    batch_size_t = 32
    batch_count = 0
    batch_pack = np.zeros((batch_size_t , win_size, win_size, 3), dtype='float32')
    batch_coords = []
    idx_counter = 0
    homogen_parts = {}
    with open(res_filename_winsize + res_fname_suffix, 'w') as file:
        for i in img_list:
            if(batch_count < batch_size_t ):
                im = Image.open(i)
                batch_pack[batch_count] = np.array(im)*1./255.
                batch_coords.append(img_coords[idx_counter])
                batch_count += 1
                idx_counter += 1
                im.close()
            else:
                pred_res = model.predict(batch_pack,batch_size=batch_size_t )
                
                for j in range(pred_res.shape[0]):
                    i_idx = batch_coords[j][0]
                    j_idx = batch_coords[j][1]
                   #print(i_idx, j_idx)
                    max_idx = np.argmax(pred_res[j])
                    #if max_idx == 0:
                       # max_idx = 1
                   # else:
                        #max_idx = 0
                    homogen_parts[(i_idx,height - j_idx)]=max_idx
                    #file.write(str(i_idx) + " " + str( height - j_idx ) + " " + str(max_idx) + "\n")
                    #for k in range(win_size):
                        #for l in range(win_size):
                            #class_marks[i_idx + k, j_idx + l] = max_idx
                    #draw.polygon([(i_idx, j_idx), (i_idx + win_size, j_idx), (i_idx + win_size, j_idx+ win_size), (i_idx, j_idx + win_size)], outline=t_color[max_idx], fill=t_color[max_idx])
                batch_count = 0
                batch_coords = []
                im = Image.open(i)
                #print(i)
                #print(img_coords[idx_counter])
                batch_pack[batch_count] = np.array(im)*1./255.
                batch_coords.append(img_coords[idx_counter])
                batch_count += 1
                idx_counter += 1
                im.close()
        for i in homogen_parts:
            i_idx = i[0]
            j_idx = height - i[1]
            elem_amount = 0
            summ = 0
            
            if (i_idx-step_len, j_idx) in homogen_parts:
                elem_amount += 1
                summ += homogen_parts[(i_idx-step_len, j_idx)]
            if (i_idx + step_len, j_idx) in homogen_parts:
                elem_amount += 1
                summ += homogen_parts[(i_idx + step_len, j_idx)]
            if (i_idx, j_idx-step_len) in homogen_parts:
                elem_amount += 1
                summ += homogen_parts[(i_idx, j_idx-step_len)]
            if (i_idx , j_idx+ step_len) in homogen_parts:
                elem_amount += 1
                summ += homogen_parts[(i_idx , j_idx + step_len)]

            if elem_amount == 4:
                if summ == 0 :#or summ == 1:
                    homogen_parts.update({(i_idx, j_idx): 0})
                if summ == 4:
                    homogen_parts.update({(i_idx, j_idx): 1})
            if elem_amount == 3:
                if summ == 0 :
                    homogen_parts.update({(i_idx, j_idx): 0})
                if summ == 3 :
                    homogen_parts.update({(i_idx, j_idx): 1})
            
        #print(homogen_parts)
        #print( homogen_parts.get((0, 0)))
        for i in homogen_parts:
            i_idx = i[0]
            j_idx = height - i[1]
            file.write(str(i_idx) + " " + str(  i[1] ) + " " + str(homogen_parts[(i_idx,  i[1])]) + "\n")
            for k in range(win_size):
                for l in range(win_size):
                    class_marks[i_idx + k, j_idx + l] = homogen_parts[(i_idx,  i[1])]
            draw.polygon([(i_idx, j_idx), (i_idx + win_size, j_idx), (i_idx + win_size, j_idx+ win_size), (i_idx, j_idx + win_size)], outline=t_color[homogen_parts[(i_idx,  i[1])]], fill=t_color[homogen_parts[(i_idx,  i[1])]])
    file.close()
    img.save(result_fileName + filename_suffix)
    img.close()
    return class_marks

In [109]:
model1=def_cnn_model_7(height_et,width_et,depth_et)
msize=16
param1 = 80
model1=train_CNN_model_ex(model1,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model1, "_7_1.BMP","_7_1.txt")

0 [0.41716739535331726]
328/328 [==============================] - 2s 5ms/step - loss: 0.5675 - accuracy: 0.6944 - val_loss: 0.6679 - val_accuracy: 0.4528
1 [0.6244634985923767]
328/328 [==============================] - 1s 4ms/step - loss: 0.5235 - accuracy: 0.7127 - val_loss: 0.6033 - val_accuracy: 0.7476
2 [0.7682403326034546]
328/328 [==============================] - 1s 4ms/step - loss: 0.5098 - accuracy: 0.7215 - val_loss: 0.5574 - val_accuracy: 0.7661
3 [0.7686695456504822]
328/328 [==============================] - 2s 5ms/step - loss: 0.5055 - accuracy: 0.7230 - val_loss: 0.5547 - val_accuracy: 0.7837
4 [0.7283262014389038]
5 [0.7682403326034546]
6 [0.7682403326034546]
7 [0.7768240571022034]
328/328 [==============================] - 2s 5ms/step - loss: 0.4839 - accuracy: 0.7351 - val_loss: 0.4935 - val_accuracy: 0.7863
8 [0.7914162874221802]
328/328 [==============================] - 2s 5ms/step - loss: 0.4815 - accuracy: 0.7351 - val_loss: 0.5123 - val_accuracy: 0.7876
9 [0.7

1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [110]:
model=def_cnn_model_9(height_et,width_et,depth_et)
msize=16
param1 = 80
model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model, "_9.BMP", "_9.txt")

0 [0.43261802196502686]
328/328 [==============================] - 1s 4ms/step - loss: 0.5657 - accuracy: 0.7053 - val_loss: 0.6618 - val_accuracy: 0.4262
1 [0.43304720520973206]
328/328 [==============================] - 1s 4ms/step - loss: 0.5321 - accuracy: 0.7137 - val_loss: 0.6494 - val_accuracy: 0.4335
2 [0.43304720520973206]
3 [0.43347638845443726]
328/328 [==============================] - 1s 4ms/step - loss: 0.5154 - accuracy: 0.7149 - val_loss: 0.6162 - val_accuracy: 0.4361
4 [0.43218883872032166]
5 [0.43433475494384766]
328/328 [==============================] - 1s 4ms/step - loss: 0.5040 - accuracy: 0.7122 - val_loss: 0.6494 - val_accuracy: 0.4361
6 [0.43476393818855286]
328/328 [==============================] - 2s 5ms/step - loss: 0.4950 - accuracy: 0.7190 - val_loss: 0.5910 - val_accuracy: 0.4356
7 [0.43648070096969604]
328/328 [==============================] - 1s 4ms/step - loss: 0.4942 - accuracy: 0.7170 - val_loss: 0.5915 - val_accuracy: 0.4365
8 [0.43390557169914246

1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 10ms/step


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [111]:
model=def_cnn_model_10(height_et,width_et,depth_et)
msize=16
param1 = 50
model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model, "_10_1.BMP","_10_1.txt")

0 [0.42060086131095886]
328/328 [==============================] - 6s 18ms/step - loss: 0.5715 - accuracy: 0.6800 - val_loss: 0.6749 - val_accuracy: 0.4708
1 [0.43433475494384766]
328/328 [==============================] - 6s 19ms/step - loss: 0.5246 - accuracy: 0.7014 - val_loss: 0.6360 - val_accuracy: 0.4747
2 [0.43047210574150085]
3 [0.43004292249679565]
4 [0.43347638845443726]
5 [0.694420576095581]
328/328 [==============================] - 6s 17ms/step - loss: 0.4974 - accuracy: 0.7182 - val_loss: 0.5913 - val_accuracy: 0.7309
6 [0.6472102999687195]
7 [0.7609441876411438]
328/328 [==============================] - 6s 17ms/step - loss: 0.4912 - accuracy: 0.7273 - val_loss: 0.5857 - val_accuracy: 0.7618
8 [0.7497854232788086]
9 [0.7618025541305542]
328/328 [==============================] - 6s 17ms/step - loss: 0.4920 - accuracy: 0.7249 - val_loss: 0.5793 - val_accuracy: 0.7691
10 [0.7214592099189758]
11 [0.7223175764083862]
12 [0.7248926758766174]
13 [0.6991416215896606]
14 [0.7304

1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 11ms/step


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [112]:
model=def_cnn_model_10(height_et,width_et,depth_et)
msize=16
param1 = 80
model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model, "_10.BMP","_10.txt")


0 [0.3905579447746277]
328/328 [==============================] - 1s 4ms/step - loss: 0.5813 - accuracy: 0.6616 - val_loss: 0.6650 - val_accuracy: 0.5210
1 [0.46394848823547363]
328/328 [==============================] - 2s 4ms/step - loss: 0.5108 - accuracy: 0.7040 - val_loss: 0.6611 - val_accuracy: 0.5155
2 [0.5974248647689819]
328/328 [==============================] - 1s 5ms/step - loss: 0.4953 - accuracy: 0.7142 - val_loss: 0.6292 - val_accuracy: 0.6991
3 [0.5463519096374512]
4 [0.7570815682411194]
328/328 [==============================] - 2s 5ms/step - loss: 0.4872 - accuracy: 0.7188 - val_loss: 0.6007 - val_accuracy: 0.7395
5 [0.6309012770652771]
6 [0.7523605227470398]
7 [0.736480712890625]
8 [0.7579399347305298]
328/328 [==============================] - 2s 5ms/step - loss: 0.4774 - accuracy: 0.7280 - val_loss: 0.5880 - val_accuracy: 0.7330
9 [0.7334764003753662]
10 [0.794420599937439]
328/328 [==============================] - 1s 4ms/step - loss: 0.4690 - accuracy: 0.7354 - v

1/1 [==============================] - 0s 4ms/step


1/1 [==============================] - 0s 11ms/step


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [113]:
model=def_cnn_model_11(height_et,width_et,depth_et)
msize=16
param1 = 80
model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model, "_11.BMP","_11.txt")

0 [0.44592273235321045]
328/328 [==============================] - 2s 5ms/step - loss: 0.5681 - accuracy: 0.6919 - val_loss: 0.7107 - val_accuracy: 0.4828
1 [0.7326180338859558]
328/328 [==============================] - 2s 5ms/step - loss: 0.5074 - accuracy: 0.7113 - val_loss: 0.6414 - val_accuracy: 0.6318
2 [0.5952789783477783]
3 [0.753648042678833]
328/328 [==============================] - 2s 6ms/step - loss: 0.4911 - accuracy: 0.7215 - val_loss: 0.5751 - val_accuracy: 0.7159
4 [0.7416309118270874]
5 [0.7347639203071594]
6 [0.6879828572273254]
7 [0.7656652331352234]
328/328 [==============================] - 2s 6ms/step - loss: 0.4797 - accuracy: 0.7294 - val_loss: 0.5558 - val_accuracy: 0.7665
8 [0.6789699792861938]
9 [0.7742488980293274]
328/328 [==============================] - 2s 5ms/step - loss: 0.4735 - accuracy: 0.7310 - val_loss: 0.5895 - val_accuracy: 0.7017
10 [0.7725321650505066]
11 [0.7862660884857178]
328/328 [==============================] - 2s 6ms/step - loss: 0.47

1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 16ms/step


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [114]:
model=def_cnn_model_11(height_et,width_et,depth_et)
msize=16
param1 = 50
model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model, "_11_1.BMP", "_11_1.txt")


0 [0.42961373925209045]
328/328 [==============================] - 2s 6ms/step - loss: 0.5539 - accuracy: 0.6863 - val_loss: 0.6153 - val_accuracy: 0.6567
1 [0.6618025898933411]
328/328 [==============================] - 2s 5ms/step - loss: 0.5033 - accuracy: 0.7125 - val_loss: 0.5736 - val_accuracy: 0.7601
2 [0.7527896761894226]
328/328 [==============================] - 1s 4ms/step - loss: 0.4914 - accuracy: 0.7186 - val_loss: 0.5586 - val_accuracy: 0.7678
3 [0.7390558123588562]
4 [0.7622317671775818]
328/328 [==============================] - 2s 5ms/step - loss: 0.4800 - accuracy: 0.7300 - val_loss: 0.5432 - val_accuracy: 0.7897
5 [0.7729613780975342]
328/328 [==============================] - 2s 5ms/step - loss: 0.4741 - accuracy: 0.7299 - val_loss: 0.5349 - val_accuracy: 0.7794
6 [0.7824034094810486]
328/328 [==============================] - 2s 5ms/step - loss: 0.4696 - accuracy: 0.7361 - val_loss: 0.5316 - val_accuracy: 0.7854
7 [0.7875536680221558]
328/328 [====================

1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [115]:
model=def_cnn_model_12(height_et,width_et,depth_et)
msize=16
param1 = 80
model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model, "_12.BMP", "_12.txt")

0 [0.43004292249679565]
328/328 [==============================] - 2s 6ms/step - loss: 0.5702 - accuracy: 0.6895 - val_loss: 0.6182 - val_accuracy: 0.6605
1 [0.716738224029541]
328/328 [==============================] - 2s 5ms/step - loss: 0.5400 - accuracy: 0.6983 - val_loss: 0.6094 - val_accuracy: 0.6738
2 [0.6733905673027039]
3 [0.5248926877975464]
4 [0.7579399347305298]
328/328 [==============================] - 2s 5ms/step - loss: 0.4837 - accuracy: 0.7278 - val_loss: 0.5621 - val_accuracy: 0.7446
5 [0.7935622334480286]
328/328 [==============================] - 2s 5ms/step - loss: 0.4802 - accuracy: 0.7291 - val_loss: 0.5496 - val_accuracy: 0.7614
6 [0.7815450429916382]
7 [0.7896995544433594]
8 [0.7278969883918762]
9 [0.7751073241233826]
10 [0.7884120345115662]
11 [0.7836909890174866]
12 [0.7849785685539246]
13 [0.7377682328224182]
14 [0.7896995544433594]
15 [0.7875536680221558]
16 [0.7922746539115906]
17 [0.7879828214645386]
18 [0.7965665459632874]
328/328 [=====================

1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 3ms/step


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [116]:
model=def_cnn_model_12(height_et,width_et,depth_et)
msize=16
param1 = 50
model=train_CNN_model_ex(model,train_x_data,train_y_data,dlin,msize, param1,10,0.1)
classify_img_by_batch_res_UDP_modified(model, "_12_1.BMP","_12_1.txt")


0 [0.43047210574150085]
328/328 [==============================] - 2s 6ms/step - loss: 0.5455 - accuracy: 0.6983 - val_loss: 0.6224 - val_accuracy: 0.5970
1 [0.5562231540679932]
328/328 [==============================] - 2s 5ms/step - loss: 0.4973 - accuracy: 0.7167 - val_loss: 0.5986 - val_accuracy: 0.6648
2 [0.7008583545684814]
328/328 [==============================] - 2s 6ms/step - loss: 0.4860 - accuracy: 0.7235 - val_loss: 0.5924 - val_accuracy: 0.7103
3 [0.778969943523407]
328/328 [==============================] - 2s 7ms/step - loss: 0.4739 - accuracy: 0.7333 - val_loss: 0.5560 - val_accuracy: 0.7481
4 [0.7639485001564026]
5 [0.7836909890174866]
328/328 [==============================] - 2s 6ms/step - loss: 0.4658 - accuracy: 0.7378 - val_loss: 0.5371 - val_accuracy: 0.7910
6 [0.793133020401001]
328/328 [==============================] - 2s 6ms/step - loss: 0.4633 - accuracy: 0.7391 - val_loss: 0.5785 - val_accuracy: 0.7309
7 [0.704291820526123]
8 [0.766094446182251]
9 [0.79871

1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 12ms/step


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])